In [6]:

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os, hashlib, base64, uuid
import seaborn as sns
import yaml
import json
from typing import Dict, List, Any
import random

plt.style.use('dark_background')
sns.set_palette("viridis")
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = 11

def domino_short_id(length=8):
    user = os.environ.get("DOMINO_USER_NAME", "demo_user")
    project = os.environ.get("DOMINO_PROJECT_ID", "demo_project")
    digest = hashlib.sha256(f"{user}/{project}".encode()).digest()
    encoded = base64.urlsafe_b64encode(digest).decode("utf-8").rstrip("=")
    return f"{user}_{encoded[:length]}"

def generate_quick_data(n=500):  # Reduced size for speed
    """Generate quick fake data for model training"""
    np.random.seed(np.random.randint(1, 1000))
    X = pd.DataFrame({
        'feature_1': np.random.randn(n),
        'feature_2': np.random.randn(n),
        'feature_3': np.random.randn(n),
        'feature_4': np.random.randn(n)
    })
    y = X.sum(axis=1) + np.random.randn(n) * 0.1
    return X, y

def create_simple_plot(model_name, metric_value, metric_name="Model Health"):
    """Create a simple performance plot"""
    fig, ax = plt.subplots(figsize=(8, 6), facecolor='black')
    
    # Simple bar chart showing model health
    ax.bar([model_name], [metric_value], color='cyan', alpha=0.7)
    ax.set_ylim(0, 100)
    ax.set_ylabel(metric_name, color='white', fontsize=12)
    ax.set_title(f'{model_name}\nPERFORMANCE METRIC', color='cyan', fontsize=14, weight='bold')
    ax.set_facecolor('black')
    
    for spine in ax.spines.values():
        spine.set_color('white')
    ax.tick_params(colors='white')
    
    plt.tight_layout()
    chart_path = f"/tmp/{model_name.replace(' ', '_')}_performance.png"
    plt.savefig(chart_path, dpi=150, bbox_inches='tight', facecolor='black')
    plt.close()
    return chart_path

# Define all models from the dashboard with their metadata
DASHBOARD_MODELS = [
    {
        "name": "Curve Bootstrapping Engine",
        "type": "Interest Rate Curves - Treasury & Swap",
        "owner": "rjohnson",
        "owner_initials": "RJ",
        "status": "production",
        "risk_class": "P1",
        "risk_tooltip": "<10min downtime",
        "model_health": 99.2,
        "active_development": ["spline smoothing", "SOFR-first rebuild"],
        "findings": [
            {"id": "MRM-2024-089", "type": "Calibration drift", "severity": "critical", "age": 47},
            {"id": "VAL-2024-156", "type": "Back-testing gaps", "severity": "high", "age": 23}
        ],
        "upstream_deps": ["Market Data Feed", "Fed H.15 Rates"],
        "downstream_deps": ["Interest Rate Swap Pricing", "FX Forward Curve", "Hedge Effectiveness Testing"],
        "governance_bundle": "SR 11-7 Model Risk Management",
        "last_run_minutes": 10,
        "runs_per_week": 2016,  # Every 5 minutes
        "model_class": "rates"
    },
    {
        "name": "Fixed Income Arbitrage",
        "type": "US Treasury Yield Curve",
        "owner": "mrodriguez",
        "owner_initials": "MR",
        "status": "production",
        "risk_class": "P2",
        "risk_tooltip": "<1hr downtime",
        "model_health": 97.8,
        "active_development": ["duration matching", "repo funding"],
        "findings": [
            {"id": "AUD-2024-203", "type": "Documentation update", "severity": "medium", "age": 12}
        ],
        "upstream_deps": ["Curve Bootstrapping Engine", "Bond Price Feed"],
        "downstream_deps": ["Daily PnL Attribution - Fixed Income"],
        "governance_bundle": "FINRA Fixed Income Guidelines",
        "last_run_minutes": 15,
        "runs_per_week": 672,
        "model_class": "rates"
    },
    {
        "name": "FX Forward Curve",
        "type": "FX Risk - Currency Hedging",
        "owner": "athompson",
        "owner_initials": "AT",
        "status": "production",
        "risk_class": "P2",
        "risk_tooltip": "<1hr downtime",
        "model_health": 92.1,
        "active_development": ["studentsT distribution", "SOFR transition"],
        "findings": [],
        "upstream_deps": ["Curve Bootstrapping Engine", "FX Spot Rates"],
        "downstream_deps": ["Hedge Effectiveness Testing", "Multi-Asset Options Pricing"],
        "governance_bundle": "FX Risk Framework 2024",
        "last_run_minutes": 30,
        "runs_per_week": 336,
        "model_class": "fx"
    },
    {
        "name": "Daily PnL Attribution - Equity",
        "type": "Performance Attribution - Equity Trading",
        "owner": "lthomas",
        "owner_initials": "LT",
        "status": "production",
        "risk_class": "P3",
        "risk_tooltip": "<4hr downtime",
        "model_health": 95.7,
        "active_development": ["sector attribution", "real-time updates"],
        "findings": [
            {"id": "MRM-2024-134", "type": "Missing dependency", "severity": "high", "age": 31},
            {"id": "QUA-2024-078", "type": "Performance review", "severity": "medium", "age": 8}
        ],
        "upstream_deps": ["Multi-Asset Options Pricing", "Equity Prices", "Risk Model"],
        "downstream_deps": ["Trading Desk Reports", "Risk Dashboard"],
        "governance_bundle": "Equity Trading Policies",
        "last_run_minutes": 60,
        "runs_per_week": 7,  # Daily
        "model_class": "pnl"
    },
    {
        "name": "Options Market Making v2.1",
        "type": "SPX Index Options",
        "owner": "jkim",
        "owner_initials": "JK",
        "status": "validation",
        "risk_class": "P1",
        "risk_tooltip": "<10min downtime",
        "model_health": 89.6,
        "active_development": ["vol surface calibration", "greeks optimization"],
        "findings": [
            {"id": "VAL-2024-198", "type": "Vol surface validation", "severity": "critical", "age": 15}
        ],
        "upstream_deps": ["Curve Bootstrapping Engine", "SPX Vol Surface", "Equity Prices"],
        "downstream_deps": ["Multi-Asset Options Pricing"],
        "governance_bundle": "Options Trading Validation",
        "last_run_minutes": 60,
        "runs_per_week": 168,
        "model_class": "options"
    },
    {
        "name": "Covariance GARCH (1,1)",
        "type": "Volatility Modeling - Fixed Income",
        "owner": "sarah_chen",
        "owner_initials": "SC",
        "status": "production",
        "risk_class": "P2",
        "risk_tooltip": "<1hr downtime",
        "model_health": 94.2,
        "active_development": ["vol-smile calibration", "performance opt"],
        "findings": [
            {"id": "MRM-2024-156", "type": "Parameter sensitivity", "severity": "medium", "age": 19}
        ],
        "upstream_deps": ["Bond Price History", "Market Vol Data"],
        "downstream_deps": ["VineCopula", "Multi-Asset Options Pricing", "Risk Scenarios"],
        "governance_bundle": "Volatility Model Framework",
        "last_run_minutes": 120,
        "runs_per_week": 84,
        "model_class": "volatility"
    },
    {
        "name": "Interest Rate Swap Pricing",
        "type": "Fixed Income - Swap Valuation",
        "owner": "clee",
        "owner_initials": "CL",
        "status": "backtesting",
        "risk_class": "P2",
        "risk_tooltip": "<1hr downtime",
        "model_health": 93.8,
        "active_development": ["CSA discounting", "OIS curves"],
        "findings": [
            {"id": "VAL-2024-167", "type": "CSA implementation", "severity": "high", "age": 29}
        ],
        "upstream_deps": ["Curve Bootstrapping Engine", "Credit Spreads"],
        "downstream_deps": ["Multi-Asset Options Pricing", "Daily PnL Attribution - Fixed Income"],
        "governance_bundle": "ISDA Swap Valuation Standards",
        "last_run_minutes": 120,
        "runs_per_week": 84,
        "model_class": "rates"
    },
    {
        "name": "Bias Detection Engine",
        "type": "Model Monitoring - Fairness",
        "owner": "dpark",
        "owner_initials": "DP",
        "status": "development",
        "risk_class": "P4",
        "risk_tooltip": "<2 days downtime",
        "model_health": 0,  # In development
        "active_development": ["fairness metrics", "drift detection"],
        "findings": [],
        "upstream_deps": ["Model Predictions", "Demographic Data"],
        "downstream_deps": ["Model Risk Dashboard", "Compliance Reports"],
        "governance_bundle": "AI Ethics Framework",
        "last_run_minutes": 180,
        "runs_per_week": 14,
        "model_class": "monitoring"
    },
    {
        "name": "Hedge Effectiveness Testing",
        "type": "Derivatives Risk - ASC 815",
        "owner": "dkim",
        "owner_initials": "DK",
        "status": "production",
        "risk_class": "P2",
        "risk_tooltip": "<1hr downtime",
        "model_health": 98.1,
        "active_development": ["shortcut method", "regression analysis"],
        "findings": [
            {"id": "AUD-2024-089", "type": "Quarterly review", "severity": "low", "age": 6}
        ],
        "upstream_deps": ["FX Forward Curve", "Interest Rate Swap Pricing", "Hedge Portfolio"],
        "downstream_deps": ["Accounting Reports", "Audit Documentation"],
        "governance_bundle": "ASC 815 Compliance",
        "last_run_minutes": 240,
        "runs_per_week": 42,
        "model_class": "hedge"
    },
    {
        "name": "CCAR Stress Testing",
        "type": "Regulatory Capital - Stress Testing",
        "owner": "ewilson",
        "owner_initials": "EW",
        "status": "production",
        "risk_class": "P0",
        "risk_tooltip": "P0 - Critical",
        "model_health": 96.4,
        "active_development": ["update DFAST scenarios", "climate stress tests"],
        "findings": [
            {"id": "REG-2024-012", "type": "Fed scenario update", "severity": "critical", "age": 52},
            {"id": "MRM-2024-203", "type": "Climate stress test", "severity": "high", "age": 38}
        ],
        "upstream_deps": ["VineCopula", "Covariance GARCH (1,1)", "Loan Portfolio", "Fed Scenarios"],
        "downstream_deps": ["Capital Planning", "Fed Submission", "Board Reports"],
        "governance_bundle": "CCAR 2024 - Fed Approved",
        "last_run_minutes": 360,
        "runs_per_week": 7,
        "model_class": "regulatory"
    },
    {
        "name": "Multi-Asset Options Pricing",
        "type": "Derivatives Pricing - Black-Scholes Extensions",
        "owner": "nshah",
        "owner_initials": "NS",
        "status": "failed",
        "risk_class": "P0",
        "risk_tooltip": "P0 - Critical",
        "model_health": 0,  # Failed
        "active_development": ["vol surface fix", "MC acceleration"],
        "findings": [
            {"id": "CRIT-2024-001", "type": "Vol surface failure", "severity": "critical", "age": 3},
            {"id": "MRM-2024-234", "type": "Monte Carlo issues", "severity": "critical", "age": 8},
            {"id": "VAL-2024-145", "type": "Greek calculations", "severity": "high", "age": 21}
        ],
        "upstream_deps": ["Options Market Making v2.1", "Interest Rate Swap Pricing", "FX Forward Curve", "Covariance GARCH (1,1)"],
        "downstream_deps": ["Daily PnL Attribution - Equity", "Options Trading Desk", "Risk Reports"],
        "governance_bundle": "Options Pricing Framework",
        "last_run_minutes": 480,
        "runs_per_week": 0,  # Failed
        "model_class": "options"
    },
    {
        "name": "VineCopula",
        "type": "Dependence Modeling - Multi-Asset",
        "owner": "mrodriguez",
        "owner_initials": "MR",
        "status": "production",
        "risk_class": "P0",
        "risk_tooltip": "P0 - Critical",
        "model_health": 91.8,
        "active_development": ["tail-risk modeling", "archimedean copulas"],
        "findings": [
            {"id": "MRM-2024-178", "type": "Tail risk calibration", "severity": "medium", "age": 14}
        ],
        "upstream_deps": ["Covariance GARCH (1,1)", "Multi-Asset Returns"],
        "downstream_deps": ["CCAR Stress Testing", "Portfolio Risk", "Scenario Generation"],
        "governance_bundle": "SR 11-7 Model Risk Management",
        "last_run_minutes": 720,
        "runs_per_week": 21,
        "model_class": "risk"
    },
    {
        "name": "Credit Fraud Detection",
        "type": "Machine Learning - Transaction Monitoring",
        "owner": "abaker",
        "owner_initials": "AB",
        "status": "production",
        "risk_class": "P1",
        "risk_tooltip": "<10min downtime",
        "model_health": 97.3,
        "active_development": ["false positive reduction", "real-time scoring"],
        "findings": [
            {"id": "MRM-2024-089", "type": "False positive rate", "severity": "high", "age": 25},
            {"id": "AUD-2024-156", "type": "Model explainability", "severity": "medium", "age": 11}
        ],
        "upstream_deps": ["Transaction Data", "Customer Profiles", "Bias Detection Engine"],
        "downstream_deps": ["Fraud Alerts", "Investigation Queue", "Compliance Reports"],
        "governance_bundle": "BSA/AML Compliance Framework",
        "last_run_minutes": 840,
        "runs_per_week": 10080,  # Real-time
        "model_class": "fraud"
    },
    {
        "name": "Daily PnL Attribution - Fixed Income",
        "type": "Performance Attribution - Bond Trading",
        "owner": "mrodriguez",
        "owner_initials": "MR",
        "status": "production",
        "risk_class": "P3",
        "risk_tooltip": "<4hr downtime",
        "model_health": 96.1,
        "active_development": ["curve attribution", "credit spread decomposition"],
        "findings": [],
        "upstream_deps": ["Fixed Income Arbitrage", "Interest Rate Swap Pricing"],
        "downstream_deps": ["Trading Desk Reports", "Risk Dashboard"],
        "governance_bundle": "Fixed Income Trading Policies",
        "last_run_minutes": 45,
        "runs_per_week": 7,
        "model_class": "pnl"
    },
    {
        "name": "LogNormal Swaption Pricing",
        "type": "Interest Rate Derivatives - Swaptions",
        "owner": "clee",
        "owner_initials": "CL",
        "status": "production",
        "risk_class": "P2",
        "risk_tooltip": "<1hr downtime",
        "model_health": 95.3,
        "active_development": ["SABR calibration", "negative rates handling"],
        "findings": [
            {"id": "MRM-2024-167", "type": "Negative rate scenarios", "severity": "medium", "age": 17}
        ],
        "upstream_deps": ["Curve Bootstrapping Engine", "Swaption Vol Surface"],
        "downstream_deps": ["Multi-Asset Options Pricing", "Swaption Trading"],
        "governance_bundle": "Swaption Valuation Standards",
        "last_run_minutes": 90,
        "runs_per_week": 112,
        "model_class": "rates"
    },
    {
        "name": "Total Return Swap Pricing",
        "type": "Equity Derivatives - TRS",
        "owner": "lthomas",
        "owner_initials": "LT",
        "status": "production",
        "risk_class": "P2",
        "risk_tooltip": "<1hr downtime",
        "model_health": 94.8,
        "active_development": ["funding cost optimization", "dividend modeling"],
        "findings": [],
        "upstream_deps": ["Equity Prices", "Funding Curves", "Dividend Forecasts"],
        "downstream_deps": ["Equity Derivatives Desk", "Risk Reports"],
        "governance_bundle": "Equity Derivatives Framework",
        "last_run_minutes": 75,
        "runs_per_week": 168,
        "model_class": "equity_derivatives"
    },
    {
        "name": "Futures Pricing Engine",
        "type": "Multi-Asset - Futures Valuation",
        "owner": "jkim",
        "owner_initials": "JK",
        "status": "production",
        "risk_class": "P1",
        "risk_tooltip": "<10min downtime",
        "model_health": 98.7,
        "active_development": ["basis risk modeling", "roll optimization"],
        "findings": [],
        "upstream_deps": ["Spot Prices", "Storage Costs", "Interest Rates"],
        "downstream_deps": ["Futures Trading Desk", "Margin Calculations"],
        "governance_bundle": "Futures Trading Compliance",
        "last_run_minutes": 5,
        "runs_per_week": 4032,
        "model_class": "futures"
    }
]

def generate_model_specific_metrics(model: Dict[str, Any]) -> Dict[str, float]:
    """Generate model-specific metrics based on model type"""
    base_metrics = {
        "rmse": np.random.uniform(0.01, 0.05) if model["status"] == "production" else np.random.uniform(0.05, 0.15),
        "r2_score": model["model_health"] / 100 if model["model_health"] > 0 else np.random.uniform(0.7, 0.85),
        "mae": np.random.uniform(0.005, 0.03),
        "mape": np.random.uniform(0.01, 0.05),
        "execution_time_ms": np.random.uniform(50, 500)
    }
    
    # Add model class specific metrics
    if model["model_class"] == "rates":
        base_metrics.update({
            "dv01": np.random.uniform(1000, 50000),
            "convexity": np.random.uniform(10, 1000),
            "curve_fit_rmse": np.random.uniform(0.1, 2.0)
        })
    elif model["model_class"] == "volatility":
        base_metrics.update({
            "vol_forecast_accuracy": np.random.uniform(0.85, 0.98),
            "garch_alpha": np.random.uniform(0.05, 0.15),
            "garch_beta": np.random.uniform(0.8, 0.95),
            "persistence": np.random.uniform(0.9, 0.99)
        })
    elif model["model_class"] == "options":
        base_metrics.update({
            "delta": np.random.uniform(0.3, 0.7),
            "gamma": np.random.uniform(0.01, 0.05),
            "theta": np.random.uniform(-0.1, -0.01),
            "vega": np.random.uniform(0.1, 0.5),
            "implied_vol_rmse": np.random.uniform(0.001, 0.01)
        })
    elif model["model_class"] == "pnl":
        base_metrics.update({
            "pnl_explained_pct": model["model_health"],
            "factor_r2": np.random.uniform(0.8, 0.95),
            "unexplained_pnl_usd": np.random.uniform(100000, 5000000),
            "attribution_accuracy": model["model_health"] / 100
        })
    elif model["model_class"] == "fraud":
        base_metrics.update({
            "precision": np.random.uniform(0.92, 0.98),
            "recall": np.random.uniform(0.88, 0.95),
            "f1_score": np.random.uniform(0.90, 0.96),
            "false_positive_rate": np.random.uniform(0.01, 0.03),
            "auc_roc": np.random.uniform(0.94, 0.99)
        })
    elif model["model_class"] == "regulatory":
        base_metrics.update({
            "stress_scenario_count": 3,
            "capital_adequacy_ratio": np.random.uniform(10, 15),
            "tier1_ratio": np.random.uniform(11, 13),
            "leverage_ratio": np.random.uniform(4, 6)
        })
    elif model["model_class"] == "risk":
        base_metrics.update({
            "var_95": np.random.uniform(-0.05, -0.02),
            "cvar_95": np.random.uniform(-0.08, -0.03),
            "correlation_accuracy": np.random.uniform(0.85, 0.95),
            "tail_dependence": np.random.uniform(0.2, 0.4)
        })
    elif model["model_class"] == "fx":
        base_metrics.update({
            "forward_points_rmse": np.random.uniform(0.1, 5),
            "hedge_effectiveness": np.random.uniform(0.92, 0.98),
            "basis_risk": np.random.uniform(0.001, 0.01)
        })
    elif model["model_class"] == "hedge":
        base_metrics.update({
            "hedge_ratio": np.random.uniform(0.85, 1.0),
            "regression_r2": np.random.uniform(0.95, 0.99),
            "dollar_offset_ratio": np.random.uniform(0.8, 1.25),
            "prospective_effectiveness": np.random.uniform(0.9, 1.0)
        })
    
    return base_metrics

def train_fake_model(model_info: Dict[str, Any]):
    """Train a fake model with metadata matching dashboard requirements"""
    model_name = model_info["name"].replace(" ", "_")
    
    mlflow.set_experiment(f"Model_Dashboard_{domino_short_id()}")
    
    with mlflow.start_run(run_name=f"{model_name}_training") as run:
        # Generate minimal fake data
        X, y = generate_quick_data(n=200)  # Small dataset for speed
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Use simple model for speed
        if model_info["status"] == "failed":
            # Simulate a failed model
            model = None
            y_pred = np.zeros_like(y_test)
        else:
            model = LinearRegression()  # Fast model
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        # Log comprehensive metadata
        mlflow.log_params({
            "model_name": model_info["name"],
            "model_type": model_info["type"],
            "owner": model_info["owner"],
            "status": model_info["status"],
            "risk_class": model_info["risk_class"],
            "model_class": model_info["model_class"],
            "governance_bundle": model_info["governance_bundle"],
            "as_of_date": datetime.today().strftime("%Y-%m-%d"),
            "data_points": len(X_train),
            "features": list(X.columns)
        })
        
        # Log model-specific metrics
        metrics = generate_model_specific_metrics(model_info)
        mlflow.log_metrics(metrics)
        
        # Log additional dashboard metadata
        mlflow.set_tags({
            "owner_initials": model_info["owner_initials"],
            "risk_tooltip": model_info["risk_tooltip"],
            "upstream_dependencies": json.dumps(model_info["upstream_deps"]),
            "downstream_dependencies": json.dumps(model_info["downstream_deps"]),
            "active_development": json.dumps(model_info["active_development"]),
            "findings": json.dumps(model_info["findings"]),
            "last_run_minutes_ago": model_info["last_run_minutes"],
            "runs_per_week": model_info["runs_per_week"],
            "model_health": model_info["model_health"]
        })
        
        # Create simple visualization
        if model_info["status"] != "failed":
            chart_path = create_simple_plot(model_info["name"], model_info["model_health"])
            mlflow.log_artifact(chart_path, "charts")
        
        # Log model if not failed
        if model is not None:
            mlflow.sklearn.log_model(model, "model", registered_model_name=model_name)
        
        print(f"✅ {model_info['name']} | Status: {model_info['status']} | Health: {model_info['model_health']}%")
        
        # Return all metadata for YAML storage
        return {
            "run_id": run.info.run_id,
            "model_info": model_info,
            "metrics": metrics,
            "training_timestamp": datetime.now().isoformat()
        }

def save_dashboard_data(all_model_data: List[Dict[str, Any]]):
    """Save all model data to YAML artifact"""
    dashboard_data = {
        "generation_timestamp": datetime.now().isoformat(),
        "dashboard_id": domino_short_id(),
        "total_models": len(all_model_data),
        "models": {}
    }
    
    # Organize by model name
    for model_data in all_model_data:
        model_name = model_data["model_info"]["name"]
        dashboard_data["models"][model_name] = model_data
    
    # Calculate summary statistics
    dashboard_data["summary"] = {
        "models_in_production": sum(1 for m in all_model_data if m["model_info"]["status"] == "production"),
        "models_in_validation": sum(1 for m in all_model_data if m["model_info"]["status"] == "validation"),
        "models_in_development": sum(1 for m in all_model_data if m["model_info"]["status"] == "development"),
        "models_failed": sum(1 for m in all_model_data if m["model_info"]["status"] == "failed"),
        "critical_findings": sum(len([f for f in m["model_info"]["findings"] if f["severity"] == "critical"]) for m in all_model_data),
        "high_priority_findings": sum(len([f for f in m["model_info"]["findings"] if f["severity"] == "high"]) for m in all_model_data),
        "owners": list(set(m["model_info"]["owner"] for m in all_model_data))
    }
    
    # Save to YAML
    yaml_path = f"/tmp/model_dashboard_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(dashboard_data, f, default_flow_style=False, sort_keys=False)
    
    print(f"\n📊 Dashboard data saved to: {yaml_path}")
    return yaml_path

if __name__ == "__main__":
    print("🚀 Starting Model Dashboard Data Generation...")
    print(f"📝 Generating data for {len(DASHBOARD_MODELS)} models\n")
    
    all_model_data = []
    
    # Train each model
    for model_info in DASHBOARD_MODELS:
        try:
            model_data = train_fake_model(model_info)
            all_model_data.append(model_data)
        except Exception as e:
            print(f"❌ Error training {model_info['name']}: {str(e)}")
    
    # Save consolidated dashboard data
    yaml_path = save_dashboard_data(all_model_data)
    
    print("\n✨ Model Dashboard Data Generation Complete!")
    print(f"📈 MLflow UI: http://localhost:5000")
    print(f"📄 Dashboard YAML: {yaml_path}")
    print(f"🔍 Total models trained: {len(all_model_data)}")
    print(f"🏭 Production models: {sum(1 for m in all_model_data if m['model_info']['status'] == 'production')}")
    print(f"⚠️  Critical findings: {sum(len([f for f in m['model_info']['findings'] if f['severity'] == 'critical']) for m in all_model_data)}")
    
    # Also save a summary JSON for easier parsing
    summary_path = f"/tmp/model_dashboard_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(summary_path, 'w') as f:
        summary = {
            "models": [
                {
                    "name": m["model_info"]["name"],
                    "owner": m["model_info"]["owner"],
                    "status": m["model_info"]["status"],
                    "health": m["model_info"]["model_health"],
                    "risk_class": m["model_info"]["risk_class"],
                    "findings_count": len(m["model_info"]["findings"]),
                    "run_id": m["run_id"]
                }
                for m in all_model_data
            ]
        }
        json.dump(summary, f, indent=2)
    
    print(f"📊 Summary JSON: {summary_path}")

🚀 Starting Model Dashboard Data Generation...
📝 Generating data for 17 models



2025/09/11 21:02:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Curve_Bootstrapping_Engine'.
Created version '1' of model 'Curve_Bootstrapping_Engine'.


✅ Curve Bootstrapping Engine | Status: production | Health: 99.2%


2025/09/11 21:02:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Fixed_Income_Arbitrage'.
Created version '1' of model 'Fixed_Income_Arbitrage'.


✅ Fixed Income Arbitrage | Status: production | Health: 97.8%


...

2025/09/11 21:03:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Total_Return_Swap_Pricing'.
Created version '1' of model 'Total_Return_Swap_Pricing'.


✅ Total Return Swap Pricing | Status: production | Health: 94.8%


2025/09/11 21:03:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Futures Pricing Engine | Status: production | Health: 98.7%

📊 Dashboard data saved to: /tmp/model_dashboard_data_20250911_210324.yaml

✨ Model Dashboard Data Generation Complete!
📈 MLflow UI: http://localhost:5000
📄 Dashboard YAML: /tmp/model_dashboard_data_20250911_210324.yaml
🔍 Total models trained: 17
🏭 Production models: 13
⚠️  Critical findings: 5
📊 Summary JSON: /tmp/model_dashboard_summary_20250911_210324.json


Successfully registered model 'Futures_Pricing_Engine'.
Created version '1' of model 'Futures_Pricing_Engine'.
